In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import json 
from tqdm import tqdm
data_path = '../Smart Attack Algorithm/data/Car Hacking Dataset/benign_data.csv'

In [2]:
df = pd.read_csv(data_path)

In [3]:
df.head()

,Timestamp,ID,DLC,Payload,label
0,1.479121e+09,0350,8,052884666d0000a2,0
1,1.479121e+09,02c0,8,1400000000000000,0
2,1.479121e+09,0430,8,0000000000000000,0
3,1.479121e+09,04b1,8,0000000000000000,0
4,1.479121e+09,01f1,8,0000000000000000,0


In [5]:
def calculate_crc(data):
    crc = 0x0000
    poly = 0x4599  # CRC-15 polynomial

    for bit in data:
        crc ^= (int(bit) & 0x01) << 14  # XOR with the current bit shifted left by 14 bits

        for _ in range(15):
            if crc & 0x8000:
                crc = (crc << 1) ^ poly
            else:
                crc <<= 1

        crc &= 0x7FFF  # Ensure 15 bits

    return crc

In [6]:
def stuff_bits(binary_string):
    result = ''
    count = 0  # Initialize a count for consecutive 0's
    
    for bit in binary_string:
        result += bit  # Append the current bit to the result string
        if bit == '0':
            count += 1  # Increment the count if the current bit is 0
            if count == 5:
                result += '1'  # Insert a 1 after 5 consecutive 0's
                count = 0  # Reset the count after inserting the 1
        else:
            count = 0  # Reset the count if the current bit is not 0
    
    return result

In [7]:
def hex_to_bits(hex_value, num_bits):
    return bin(int(hex_value, 16))[2:].zfill(num_bits)

def convert_to_binary_string(can_id, dlc, data):
    start_of_frame = '0'
    can_id_bits = stuff_bits(hex_to_bits(can_id, 11))
    rtr_bit = '0'
    ide_bit = '0'
    control_r0_bit = '0'
    control_stuff_bit = '1'
    dlc_bits = bin(dlc)[2:].zfill(4)
    data_bits = ''.join(hex_to_bits(hex_byte, 8) for hex_byte in data)
    padding_bits = '0' * (8 * (8 - dlc))  # Fill missing data bytes with zeros
    data_bit_total = stuff_bits(data_bits+padding_bits)
    crc_bit =stuff_bits(bin(calculate_crc(start_of_frame+can_id_bits +rtr_bit+ide_bit+control_r0_bit+control_stuff_bit+ dlc_bits + data_bit_total))[2:].zfill(15))
    crc_delimiter = '1'
    ack_bit = '0'
    ack_delimiter = '1'
    end_of_frame_bits = '1'*7
    inter_Frame_spacing_bits = '1'*3
    return start_of_frame+can_id_bits +rtr_bit+ide_bit+control_r0_bit+ control_stuff_bit+dlc_bits + data_bit_total+crc_bit + crc_delimiter+ack_bit +ack_delimiter+ end_of_frame_bits+inter_Frame_spacing_bits
     


In [8]:
data_array = []

for index, data in df.iterrows():

    timestamp = data['Timestamp']
    id = data['ID']
    dlc = data['DLC']
    payload = data['Payload']

    converted_data = convert_to_binary_string(id, dlc, payload)

    data_array.append([timestamp, converted_data])


In [9]:
data_rate=512000 #512kbps
# number_of_bits_in_each_frame=114
# frame_duration=number_of_bits_in_each_frame/data_rate
total_bits_per_image=128*128
def create_image(binary_matrix):
    width = len(binary_matrix[0])
    height = len(binary_matrix)
    # pixel_mapping={'3':150,'2':100,'0':0,'1':255}
    color_mapping = {
        '3': (255, 0, 0),  # Red
        '2': (0, 255, 0),  # Blue
        '1': (255, 255, 255),  # White
        '0': (0, 0, 0)  # Black
    }
    image_data = [color_mapping[value] for row in binary_matrix for value in row]
    image = Image.new('RGB', (width, height))
    image.putdata(image_data)
    return image
def calculate_interframe_bits(data_array,timestamp_difference, data_rate,i):
    length_of_frame = len(data_array[i])
    frame_duration=length_of_frame/data_rate
    interframe_time = timestamp_difference - frame_duration
    interframe_bits = int(data_rate * interframe_time)
    return '2' * interframe_bits


In [10]:
binary_matrix = []

a = [['0' for _ in range(128)] for _ in range(128)]

i = 0
x = 0
y = 0

while i < len(data_array):
    bin_str = data_array[i][1]
    for bit in bin_str:
        a[x][y] = bit
        if y == 127:
            x += 1
            y = 0
        else:
            y += 1
        if x == 128:
            binary_matrix.append(a)
            a = [['0' for _ in range(128)] for _ in range(128)]
            x = 0
            y = 0
        
        
        
    if i < len(data_array) - 1:
        timestamp_difference = data_array[i+1][0] - data_array[i][0]
        interframe_bits = calculate_interframe_bits(data_array, timestamp_difference, data_rate,i)
        for bit in interframe_bits:
            a[x][y] = bit
            if y == 127:
                x += 1
                y = 0
            else:
                y += 1
            if x == 128:
                binary_matrix.append(a)
                a = [['0' for _ in range(128)] for _ in range(128)]
                x = 0
                y = 0
    while y < 128 and y>0 :
        a[x][y] = '3'
        if y == 127:
            x += 1
            if x == 128:
                binary_matrix.append(a)
                a = [['0' for _ in range(128)] for _ in range(128)]
                x=0
                y=0
            y=0
            break 
        y+=1    

    i += 1

In [11]:
img_matrix=[]
size_of_binary_matrix=len(binary_matrix)
for i in  range (size_of_binary_matrix):
    img_matrix.append(create_image(binary_matrix[i]))

In [12]:
print(np.array(img_matrix).shape)

/var/folders/pw/3vr89zzj1xvbzvtk1yfz9kkc0000gn/T/ipykernel_95346/2664206375.py:1: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  print(np.array(img_matrix).shape)
/var/folders/pw/3vr89zzj1xvbzvtk1yfz9kkc0000gn/T/ipykernel_95346/2664206375.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(np.array(img_matrix).shape)


(31574,)


In [14]:
print(np.array(binary_matrix).shape)

(31574, 128, 128)


In [15]:
output_path = './images/Car Hacking/benign'

for i,img in enumerate(img_matrix):
    
    img_path=os.path.join(output_path,f"image_{i+1}.png")
    
    img.save(img_path)

print("Images saved")

Images saved


In [16]:
smart_attack_path = '../Smart Attack Algorithm/data/Car Hacking Dataset/smart_output.csv'

smart_attack = pd.read_csv(smart_attack_path)

In [17]:
smart_attack['DLC'].unique()

array([8., 5., 2.])

In [18]:
attack_data = []
attack_labels = []

for index, data in smart_attack.iterrows():

    timestamp = data['Timestamp']
    id = hex(int(data['ID']))[2:]
    dlc = int(data['DLC'])
    payload = hex(int(data['Payload']))[2:]

    if len(payload) != int(dlc) * 2:
        payload =  '0' * (int(dlc) * 2 - len(payload)) + payload
    
    converted_attack_data = convert_to_binary_string(id, dlc, payload)

    attack_data.append([timestamp, converted_attack_data])
    attack_labels.append(data['label'])

In [19]:
attack_binary_matrix = []
y1=[]

a = [['0' for _ in range(128)] for _ in range(128)]
i = 0
x = 0
y = 0
flag = 0
while i < len(attack_data):
    bin_str = attack_data[i][1]
    if int(attack_labels[i]==1):
        flag =  1
        # print(flag)
    for bit in bin_str:
        a[x][y] = bit
        if y == 127:
            x += 1
            y = 0
        else:
            y += 1
        if x == 128:
            attack_binary_matrix.append(a)
            y1.append(1 if flag == 1 else 0)
            falg = 0
            a = [['0' for _ in range(128)] for _ in range(128)]
            x = 0
            y = 0
        
        
        
    if i < len(attack_data) - 1:
        timestamp_difference = attack_data[i+1][0] - attack_data[i][0]
        interframe_bits = calculate_interframe_bits(attack_data, timestamp_difference, data_rate,i)
        for bit in interframe_bits:
            a[x][y] = bit
            if y == 127:
                x += 1
                y = 0
            else:
                y += 1
            if x == 128:
                attack_binary_matrix.append(a)
                y1.append(1 if flag == 1 else 0)
                flag = 0
                a = [['0' for _ in range(128)] for _ in range(128)]
                x = 0
                y = 0
    while y < 128 and y>0 :
        a[x][y] = '3'
        if y == 127:
            x += 1
            if x == 128:
                attack_binary_matrix.append(a)
                y1.append(1 if flag == 1 else 0)
                falg = 0
                a = [['0' for _ in range(128)] for _ in range(128)]
                x=0
                y=0
            y=0
            break 
        y+=1    

    i += 1

In [17]:
print(len(attack_binary_matrix))
print(len(y1))

37616
37616


In [20]:
print(np.array(attack_binary_matrix).shape)

(37616, 128, 128)


In [84]:
##Check this!
np.unique(np.array(y1), return_counts=True)

(array([0, 1]), array([27599, 10017]))

In [21]:
images_dir = 'images/Car Hacking/'
attack_images_dir = os.path.join(images_dir, 'attack')
json_path = os.path.join(images_dir, 'labels.json')

In [24]:
attack_img_matrix=[]
labels_dict = {}
size_of_binary_matrix1=len(attack_binary_matrix)

for i in tqdm(range(size_of_binary_matrix1)):
    attack_img_matrix.append(create_image(attack_binary_matrix[i]))

for i,img in enumerate(attack_img_matrix):
    img_path=os.path.join(attack_images_dir,f"image_{i+1}.png")
    label = y1[i]
    img.save(img_path)

    labels_dict[f"image_{i+1}.png"] = label

with open(json_path, 'w') as json_file:
    json.dump(labels_dict, json_file)


# print("Images saved")
# image_array_list1 = [np.array(img) for img in attack_img_matrix]
# final_array1 = np.stack(image_array_list1)

100%|██████████| 37616/37616 [00:33<00:00, 1136.20it/s]


In [95]:
os.listdir(images_dir)

['attack', '.DS_Store', 'lables.json', 'benign']

In [97]:
with open(json_path, 'a+') as file:
    # Load the existing data
    existing_data = json.load(file)

print(existing_data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# /Users/anwesh/code/IIT-Projects/Adversarial-Attacks-and-Defenses-in-In-Vehicular-Networks/traffic-to-image/images/Car Hacking/lables.json